In [11]:
import pandas as pd 
import numpy as np 
import requests

In [12]:
endpoint = "https://datos.madrid.es/egob/catalogo/300107-0-agenda-actividades-eventos.json"

In [13]:
# Realizar la solicitud GET
response = requests.get(endpoint)

In [14]:
if response.status_code == 200:
    data = response.json()  # Convertir la respuesta a un diccionario de Python
else:
    print(f"Error en la solicitud: {response.status_code}")

In [15]:
eventos = data.get("@graph")

fecha_estancia = pd.to_datetime("2025-03-02")


dictio_eventos = {
        "id_evento": [],
        "nombre_evento": [],
        "url_evento": [],
        "codigo_postal": [],
        "direccion": [],
        "horario": [],
        "fecha_inicio": [],
        "fecha_fin": [],
        "organizacion": [],
        "id_ciudad": []
    }



for i, evento in enumerate(eventos):

    graph = data.get("@graph")

    dtstart = pd.to_datetime(graph[i].get("dtstart"), errors="coerce")
    dtend = pd.to_datetime(graph[i].get("dtend"), errors="coerce")

    if not pd.isna(dtstart) and not pd.isna(dtend): # Si las fechas no son nulas

        if (dtstart <= fecha_estancia) & (fecha_estancia <= dtend): # Si los eventos ocurren dentro de la fecha de estancia
            
            id_evento = int(graph[i].get("id", np.nan)) 
            dictio_eventos["id_evento"].append(id_evento)

            nombre_evento = graph[i].get("title", "Sin nombre") 
            dictio_eventos["nombre_evento"].append(nombre_evento)

            url_evento = graph[i].get("link", "Sin link") 
            dictio_eventos["url_evento"].append(url_evento)

            codigo_postal = int(graph[i].get("address", {}).get("area", {}).get("postal-code", 0))
            dictio_eventos["codigo_postal"].append(codigo_postal)

            direccion = graph[i].get("address", {}).get("area", {}).get("street-address", "Sin direccion") 
            dictio_eventos["direccion"].append(direccion)

            horario = graph[i].get("time", "Sin horario") if graph[i].get("time", "Sin horario") != "" else "Sin horario"
            dictio_eventos["horario"].append(horario)

            fecha_inicio = pd.to_datetime(graph[i].get("dtstart", pd.NaT)).strftime('%Y-%m-%d')
            dictio_eventos["fecha_inicio"].append(fecha_inicio)

            fecha_fin = pd.to_datetime(graph[i].get("dtend", pd.NaT)).strftime('%Y-%m-%d') 
            dictio_eventos["fecha_fin"].append(fecha_fin)

            organizacion = graph[i].get("organization", {}).get("organization-name", "Sin organizacion") 
            dictio_eventos["organizacion"].append(organizacion)

            dictio_eventos["id_ciudad"].append(1)




In [16]:
df_eventos = pd.DataFrame(dictio_eventos)
df_eventos.sample(10)

,id_evento,nombre_evento,url_evento,codigo_postal,direccion,horario,fecha_inicio,fecha_fin,organizacion,id_ciudad
29,11441198,Ciclismo por Madrid,http://www.madrid.es/sites/v/index.jsp?vgnextc...,0,Sin direccion,00:00,2024-09-01,2025-06-30,Sin organizacion,1
76,12548493,Explora y juega Matadero Madrid en Familia,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28045,PLAZA LEGAZPI 8,Sin horario,2024-10-05,2026-06-30,Matadero Madrid,1
192,12065968,Triatlón por Madrid,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28042,AVENIDA LOGROÑO 70,00:00,2024-09-01,2025-06-30,Centro Deportivo Municipal Barajas,1
92,12128186,IX Muestra de Orfebrería Contemporánea,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28012,CALLE CONCEPCION JERONIMA 15,14:00,2025-01-28,2025-03-23,Imprenta Municipal - Artes del libro,1
127,12576555,Nuestros lugares en el barrio [Mujeres Compart...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28022,PLAZA EURIPIDES 1,10:30,2025-01-09,2025-03-27,Espacio de Igualdad Lucía Sánchez Saornil. San...,1
148,12695794,Sombras de tinta,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28039,CALLE FRANCOS RODRIGUEZ 67,Sin horario,2025-02-21,2025-03-21,Biblioteca Pública Municipal Manuel Vázquez Mo...,1
177,12359163,Talleres Familiares,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28049,Carretera M-607 Km 13 L-10,12:00,2025-03-01,2025-03-29,Centro de Interpretación de la Naturaleza Mont...,1
75,12593032,Estrenos Cineteca,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28045,PLAZA LEGAZPI 8,Sin horario,2024-09-05,2025-07-31,Cineteca Madrid,1
71,12549078,Escuela de espectadores sénior,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28045,PASEO CHOPERA 14,Sin horario,2024-10-03,2025-06-25,Nave 10 Matadero. Creación Dramática Contempor...,1
41,12397727,Club de lectura en inglés 'Reading Club',http://www.madrid.es/sites/v/index.jsp?vgnextc...,28004,CALLE BARCELO 4,19:00,2024-10-07,2025-04-28,Biblioteca Pública Municipal Mario Vargas Llos...,1


In [17]:
df_eventos.isnull().sum() 


id_evento        0
nombre_evento    0
url_evento       0
codigo_postal    0
direccion        0
horario          0
fecha_inicio     0
fecha_fin        0
organizacion     0
id_ciudad        0
dtype: int64

In [18]:
df_eventos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id_evento      211 non-null    int64 
 1   nombre_evento  211 non-null    object
 2   url_evento     211 non-null    object
 3   codigo_postal  211 non-null    int64 
 4   direccion      211 non-null    object
 5   horario        211 non-null    object
 6   fecha_inicio   211 non-null    object
 7   fecha_fin      211 non-null    object
 8   organizacion   211 non-null    object
 9   id_ciudad      211 non-null    int64 
dtypes: int64(3), object(7)
memory usage: 16.6+ KB


In [19]:
df_eventos["fecha_inicio"] = pd.to_datetime(df_eventos["fecha_inicio"], format="%Y-%m-%d", errors="coerce")
df_eventos["fecha_fin"] = pd.to_datetime(df_eventos["fecha_fin"], format="%Y-%m-%d", errors="coerce")
df_eventos.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id_evento      211 non-null    int64         
 1   nombre_evento  211 non-null    object        
 2   url_evento     211 non-null    object        
 3   codigo_postal  211 non-null    int64         
 4   direccion      211 non-null    object        
 5   horario        211 non-null    object        
 6   fecha_inicio   211 non-null    datetime64[ns]
 7   fecha_fin      211 non-null    datetime64[ns]
 8   organizacion   211 non-null    object        
 9   id_ciudad      211 non-null    int64         
dtypes: datetime64[ns](2), int64(3), object(5)
memory usage: 16.6+ KB


In [20]:
df_eventos.to_pickle("../data/api/eventos_madrid.pkl")